In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.9MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [5]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix/"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [6]:
ls

 data/                 LICENSE              matrix_one/
'Hello Github.ipynb'   matrix_day_4.ipynb   README.md


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [9]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]

In [15]:
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [16]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict

  features = literal_eval(x.replace('\\"','"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()
    output_dict[key] = value
  
  return output_dict

  

df['features_parsed'] = df['features'].map(parse_features)

In [23]:
df['features_parsed'].sample(5).values

array([{'gender': 'men', 'shoe size': "men's", 'fabric material': 'leather', 'color': 'black', 'shipping weight (in pounds)': '3.8108', 'casual & dress shoe style': 'fashion trend: ankle-high synthetic boot', 'manufacturer part number': 'dr.m45001-vegan14-black720.12m', 'designer': 'dr. martens', 'brand': 'dr. martens'},
       {'material': 'synthetic', 'gender': 'men', 'shoe size': '10m', 'size': '10', 'shoe category': "men's shoes", 'color': 'royal/dark grey/silver', 'model': 'hi-m-pajo-royaldar76.10m', 'manufacturer part number': 'hi-m-pajo-royaldar76.10m', 'brand': 'hi-tec', 'age group': 'adult'},
       {'gender': 'unisex', 'shoe size': '9', 'shoe category': 'unisex shoes', 'fabric material': 'multi', 'color': 'multicolor', 'brand': 'servus', 'manufacturer_part_number': '617-22115-11', 'age group': 'adult'},
       {'shoe size': '15 / wide', 'gender': 'men', 'shoe category': "men's shoes", 'brand': 'danner', 'color': 'a-tacs'},
       {'style': 'basketball shoes', 'condition': 'ne

In [22]:
keys = set()

df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [26]:
def get_name_feats(key):
  return "feat_" + key

for key in tqdm_notebook(keys):
  df[get_name_feats(key)] = df['features_parsed'].map(lambda feats: feats[key] if key in feats else np.nan)

In [0]:
keys_stat = {}

for key in keys:
  keys_stat[key] = df[ False == df[get_name_feats(key)].isnull() ].shape[0] / df.shape[0] * 100

In [28]:
keys_stat

{'100% adidas authentic': 0.005470459518599562,
 'accessory type': 0.1422319474835886,
 'adidas': 0.005470459518599562,
 'adjustable': 0.34463894967177244,
 'age': 0.6400437636761488,
 'age end': 0.4431072210065646,
 'age gender group': 0.12582056892778995,
 'age group': 27.64770240700219,
 'age range': 0.5470459518599562,
 'age segment': 0.1422319474835886,
 'age start': 0.4431072210065646,
 'airport friendly': 0.005470459518599562,
 'alarm': 0.23522975929978115,
 'amazonbestsellersrank': 0.06017505470459519,
 'animal type': 0.005470459518599562,
 'antiscratch lens coating': 0.005470459518599562,
 'applicable': 0.02188183807439825,
 'arm': 0.6236323851203501,
 'article': 0.005470459518599562,
 'assembled in country of origin': 2.199124726477024,
 'assembled product dimensions (l x w x h)': 10.300875273522976,
 'assembled product weight': 1.5536105032822756,
 'athlete': 0.0437636761487965,
 'atpv arc rating': 0.005470459518599562,
 'attachment': 0.03282275711159737,
 'audience': 0.2352

In [32]:
{k:v for k,v in keys_stat.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_gender_cat'] = df['feat_gender'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]
df['brand'] = df.brand.map(lambda x: str(x).lower())

df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

for key in keys:
  df[get_name_feats(key) + '_cat'] = df[get_name_feats(key)].factorize()[0]

In [34]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)

run_model([ 'brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [0]:
feats_cat = [x for x in df.columns if '_cat' in x]

feats = [ 'brand_cat', 'feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat', 'feat_style_cat', 'feat_shape_cat', 'feat_metal type_cat' ]
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)

result = run_model(feats, model)

In [53]:
X = df[ feats ].values
y = df['prices_amountmin'].values
m = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
m.fit(X, y)

print(result)
perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats)

(-57.336112376479925, 4.233623862309264)


Weight,Feature
0.2558 ± 0.0080,brand_cat
0.1035 ± 0.0163,feat_material_cat
0.0249 ± 0.0039,feat_gender_cat
0.0178 ± 0.0007,feat_brand_cat
0.0128 ± 0.0004,feat_shape_cat
0.0083 ± 0.0014,feat_metal type_cat
0.0035 ± 0.0008,feat_style_cat
0.0002 ± 0.0000,feat_sport_cat


In [41]:
df['brand'].value_counts(normalize=True)

nike                      0.097210
puma                      0.033315
ralph lauren              0.028775
vans                      0.021116
new balance               0.020295
                            ...   
boconi                    0.000055
dim                       0.000055
fuel                      0.000055
under armour outerwear    0.000055
benefit wear              0.000055
Name: brand, Length: 1732, dtype: float64